This notebook guides through the process of semi-manually segmenting the background of the Transmission channel. The obtained mask sparsifies the image data significantly and is needed for computing astar distances through the microstructure. First load the Transmission channel into napari. Perform prewitt edge detection, gaussian denoising, binarize using otsu, run binary closing. Then comes the manual part where one needs to close all the edges. Once the has been done, use flood to segement the PDMS structure channels. Two masks are saved, one with the explant wel included, one with them excluded.

In [3]:
import napari
from tifffile import imread
import numpy as np

from skimage import filters
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.segmentation import flood

import pandas as pd

In [188]:
%gui qt

inp_path = '/run/media/loaloa/lbb_ssd/timelapse13_processed/'
inp_path = '/home/loaloa/Documents/timelapse13_processed/'


name = 'D11_G014'

fname = f'{name}_Transmission_compr.deflate.tif'
mask_fname = fname.replace('.tif', '_mask1.npy')
metadata_fname = fname.replace('.tif', '_metadata.csv')

tranl_file = inp_path+fname
transl_ch = imread(tranl_file)[0]

In [125]:
with napari.gui_qt():
    viewer = napari.Viewer()

In [189]:
# load transmission channel
viewer.add_image(transl_ch, name=name)
viewer.layers[name].blending='additive'
# viewer.layers['transl_ch'].contrast_limits = [.007*2**16, .0191*2**16]
viewer.layers[name].opacity = .7
viewer.layers[name].gamma = .72

## If old mask0 (or mask1) already exists, load it here. 


In [118]:
# mask1
which_mask = 'mask1'
mask_fname = f'{name}_Transmission_compr.deflate_{which_mask}.npy'
mask_file = inp_path+mask_fname

mask = np.load(mask_file)
viewer.add_labels(mask, name='inital_mask')

<Labels layer 'inital_mask' at 0x7f6fd22d3a00>

# Load from gimp segmentation 

In [180]:
# mask0
which_mask = 'mask0'
mask_fname = f'{name}_Transmission_compr.deflate_{which_mask}.tif'
mask_file = inp_path+mask_fname

mask = imread(mask_file)[:,:,0].astype(bool)
print(mask)
print(mask.shape)
viewer.add_labels(mask, name='inital_mask')

Shaped series: series shape does not match page shape
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
(1972, 3868)


<Labels layer 'inital_mask [2]' at 0x7f6f976a2fa0>

In [190]:
# edge detection
prewitt = filters.prewitt(transl_ch)
viewer.add_image(prewitt)
# perform guassian smoothing on edge filtered image
prewitt_gaussian = filters.gaussian(prewitt, sigma=1)
viewer.add_image(prewitt_gaussian)
# binarize image
prewitt_bin = prewitt > threshold_otsu(prewitt_gaussian)
viewer.add_image(prewitt_bin)
# Run this, then use the bucket tool to fill everything. 
# Play with sigma and solem diameter if things look bad.
# delete parts ofthe image where channels almost touch each other
diameter = 6
selem = morphology.selem.square(diameter) 
prewitt_bin_closed = morphology.binary_closing(prewitt_bin, selem)
viewer.add_labels(prewitt_bin_closed, name='inital_mask')

<Labels layer 'inital_mask' at 0x7f6f97664940>

In [191]:
target = (0,1900)
notes = ''
design = int(name[1:3])
CLSM_area = int(name[5:8])
dt = 31
pixelsize = 0.62

metadata = pd.Series({'target': target,
                      'notes': notes,
                      'design': design,
                      'CLSM_area': CLSM_area,
                      'dt': dt,
                      'pixelsize': pixelsize,
                })
metadata

target       (0, 1900)
notes                 
design              11
CLSM_area           14
dt                  31
pixelsize         0.62
dtype: object

In [192]:
# flood 
prewitt_bin_closed = viewer.layers['inital_mask'].data
print(prewitt_bin_closed.shape)
prewitt_bin_closed_flooded = flood(prewitt_bin_closed, target) 
# prewitt_bin_closed_flooded2 = flood(prewitt_bin_closed, (0,1720))
# prewitt_bin_closed_flooded = prewitt_bin_closed_flooded |prewitt_bin_closed_flooded2
viewer.add_labels(prewitt_bin_closed_flooded, name='before_clean')

(1972, 3868)


<Labels layer 'before_clean' at 0x7f6f9e6eb790>

In [194]:
prewitt_bin_closed_flooded = viewer.layers['before_clean'].data
selem = morphology.selem.diamond(4)
final = filters.median(prewitt_bin_closed_flooded, selem)
# final fixing
viewer.add_labels(final, name='final')

<Labels layer 'final [1]' at 0x7f6f9ea110a0>

------------------------------------

In [184]:
# final mask
mask = viewer.layers['final'].data.astype(bool)
# segment target and starts
viewer.add_labels(np.zeros_like(mask), name='goal_mask')
viewer.add_labels(np.zeros_like(mask), name='start_mask')

<Labels layer 'start_mask' at 0x7f6f7056fe50>

In [185]:
goal_mask = viewer.layers['goal_mask'].data & mask
start_mask = viewer.layers['start_mask'].data & mask

In [186]:
np.save(tranl_file.replace('.tif', '_mask1.npy'), mask)
np.save(tranl_file.replace('.tif', '_mask1_goal.npy'), start_mask)
np.save(tranl_file.replace('.tif', '_mask1_start.npy'), goal_mask)
metadata.to_csv(inp_path+metadata_fname)

In [187]:
# check
mask = np.load(tranl_file.replace('.tif', '_mask1.npy'))
viewer.add_image(mask, opacity=.5)
goal_mask = np.load(tranl_file.replace('.tif', '_mask1_goal.npy'))
viewer.add_image(goal_mask, opacity=.5)
start_mask = np.load(tranl_file.replace('.tif', '_mask1_start.npy'))
viewer.add_image(start_mask, opacity=.5)

<Image layer 'start_mask [1]' at 0x7f6f975b1220>